# Evaluating Models

Model evaluation is not just the end point of our machine learning pipeline. Before we handle any data, we want to plan ahead and use techniques and metrics that are suited for our purposes.

### <a name="1"></a> 1. Model Evaluation Applications
Let's start with a question: **"Why do we care about performance estimates at all?"**

<a name="1.1"></a>**Generalization performance** - We want to estimate the predictive performance of our model on future (unseen) data.
- Ideally, the estimated performance of a model tells how well it performs on unseen data – making predictions on future data is often the main problem we want to solve.

<a name="1.2"></a>**Model selection** - We want to increase the predictive performance by tweaking the learning algorithm and selecting the best performing model from a given hypothesis space.
- Typically, machine learning involves a lot of experimentation. Running a learning algorithm over a training dataset with different hyperparameter settings and different features will result in different models. Since we are typically interested in selecting the best-performing model from this set, we need to find a way to estimate their respective performances in order to rank them against each other.

<a name="1.3"></a>**Algorithm selection** - We want to compare different ML algorithms, selecting the best-performing one.
- We are usually not only experimenting with the one single algorithm that we think would be the “best solution” under the given circumstances. More often than not, we want to compare different algorithms to each other, oftentimes in terms of predictive and computational performance.

Although these three sub-tasks have all in common that we want to estimate the performance of a model, they all require different approaches. 

This tutorial will focus on **supervised learning**, a subcategory of machine learning where our target values are known in our available dataset.

### <a name="2"></a>2. Model Evaluation Techniques
#### <a name="2.1"></a>Holdout method (simple train/test split)
The holdout method is the simplest model evaluation technique. We take our labeled dataset and split it randomly into two parts: A **training set** and a **test set**

<img src="https://sebastianraschka.com/images/blog/2016/model-evaluation-selection-part1/testing_01.png" width="500">

Then, we fit a model to the training data and predict the labels of the test set.
<img src="https://sebastianraschka.com/images/blog/2016/model-evaluation-selection-part1/testing_02.png" width="500">

And the fraction of correct predictions constitutes our estimate of the prediction accuracy.
<img src="https://sebastianraschka.com/images/blog/2016/model-evaluation-selection-part1/testing_03.png" width="500">

We really don’t want to train and evaluate our model on the same training dataset, since it would introduce **overfitting**. In other words, we can’t tell whether the model simply memorized the training data or not, or whether it generalizes well to new, unseen data.

##### Pros:
    + Simple
    + Fast

##### Cons:
    - Not so precise estimate of out-of-sample performance comparing to more advanced techniques

### Be aware.

As it was said, you want your validation to mimic your test set as close as possible. And you can make a fair assumprion (that is not always true), that distribution of target on train and not seen data is the same. Then you have to use stratification. Stratification ensures stable distributions across split. That is more than just useful if:

    + Dataset is small
    + Dataset is unbalanced (target average for binary classification this means average target close to 0 or to 1)
    + You have multiclassification task

See example below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import data
import pandas as pd

titanic_train = pd.read_csv("/content/drive/MyDrive/projector_course_data/train_titanic.csv")

# check number of rows & columns
titanic_train.shape

In [ ]:
titanic_train.head()

In [ ]:
# split dataset to Train and Test parts
from sklearn.model_selection import train_test_split

titanic_features, titanic_label = titanic_train.drop('Survived', axis = 1), titanic_train.Survived
titanic_features_train, titanic_features_test, titanic_label_train, titanic_label_test = \
  train_test_split(titanic_features, titanic_label, test_size=0.2, random_state=42)

In [ ]:
titanic_label_train.value_counts(normalize=True)

In [ ]:
titanic_label_test.value_counts(normalize=True)

In [ ]:
# fit a model to the training data
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

model = classifier.fit(titanic_features_train, titanic_label_train)

In [ ]:
# predict the labels of the test set
predictions = model.predict(titanic_features_test)

In [ ]:
# compute prediction accuracy
from sklearn import metrics
titanic_label_test.value_counts(normalize=True)
print("Accuracy:", metrics.accuracy_score(titanic_label_test, predictions))
print("Train Accuracy:", metrics.accuracy_score(titanic_label_train, model.predict(titanic_features_train)))

In [ ]:
titanic_features_train, titanic_features_test, titanic_label_train, titanic_label_test = \
  train_test_split(titanic_features, titanic_label, test_size=0.2, random_state=42, stratify = titanic_label)

In [ ]:
titanic_label_train.value_counts(normalize=True)

In [ ]:
titanic_label_test.value_counts(normalize=True)

In [ ]:
# fit a model to the training data

classifier = LogisticRegression()


model = classifier.fit(titanic_features_train, titanic_label_train)

predictions = model.predict(titanic_features_test)

titanic_label_test.value_counts(normalize=True)
print("Accuracy:", metrics.accuracy_score(titanic_label_test, predictions))
print("Train Accuracy:", metrics.accuracy_score(titanic_label_train, model.predict(titanic_features_train)))

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/projector_course_data/test_titanic.csv")
test.set_index("PassengerId", inplace=True)
test["Survived"] = model.predict(test)
#test['Survived'].reset_index().to_csv('pred/pred.csv', index = False)

### <a name="2.2"></a>K-fold Cross-validation
K-fold Cross-validation is probably the most common technique for model evaluation and model selection. 
- We split the dataset into *K* parts and iterate over a dataset set *K* times
- In each round one part is used for validation, and the remaining *K-1* parts are merged into a training subset for model evaluation
- We compute the cross-validation performance as the arithmetic mean over the *K* performance estimates from the validation sets.
<img src="https://sebastianraschka.com/images/blog/2016/model-evaluation-selection-part3/kfold.png" width="500">

##### Pros:
    + Better estimate of out-of-sample performance than simple train/test split

##### Cons:
    - Runs "K" times slower than simple train/test split

If we have **little data** and **enough time**, it's better to always do cross-validation for a more precise estimate of performance.

In the following example we will apply k-fold cross validation for Model Selection using *GridSearchCV* function.

> #### GridSearchCV main parameters
>*sklearn.model_selection.GridSearchCV*

>**param_grid**: dict or list of dictionaries.
Dictionary with parameters names (string) as keys and lists of parameter settings to try as values, or a list of such dictionaries, in which case the grids spanned by each dictionary in the list are explored. This enables searching over any sequence of parameter settings.

>**cv**: int, cross-validation generator or an iterable, optional.
Determines the cross-validation splitting strategy.

>**scoring**: string, callable or None, default=None.
Controls what metric to apply to the estimators evaluated

### <a name="2.2"></a>LOO or Leave One Out validation
LOO validation is a corner case of K-fold cross-validation, where *K* is equal to *N* - number of examples in the dataset.  
- We split the dataset into *N* parts, where *i-th* part is the original dataset sans i-th example
- In each round i-th example is used for validation, and the remaining *N-1* examples creates a training for model 
- We compute the cross-validation performance as the arithmetic mean over the same as in K_Fold

You can use LOO validation in case you have a small dataset and/or very easy model to train


In [ ]:
# fit model 
from sklearn.model_selection import GridSearchCV

params = dict(C=[100, 10, 1, 0.1, 0.01, 0.001, 0.0001])
grid_search = GridSearchCV(classifier, param_grid=params, cv=3)

%time grid_search.fit(titanic_features_train, titanic_label_train)

In [ ]:
# Best parameters found:
grid_search.best_params_

In [ ]:
# Average accuracy over K folds for best parameters set
print("Validation Accuracy", grid_search.best_score_)

## Splitting dataset into train and validation

### Row validation. Random
This assumes that rows are independent such as loan default prediction where each row represents a client. This is not always true as if there are family members, you can assume that they also will be able to pay off a loan. Although this dependency can lead to interesting leaks/feature generation depending on whether family members were splitted to different train and test.


Another type of validation construction - is by group. Suppose you have a task to build a model to predict a weather in cities based on previous dates. Then if you know that in test set there are only new unseen cities, you should split yor dataset on train and validation such as there is no records for any city present in both train and validation.


### Time Validation

Doing **Time validation** in correct way is very important. Suppose you have a task to predict Wikipedia page viewers as in on of previous Kaggle competitions (https://www.kaggle.com/c/web-traffic-time-series-forecasting). What are possible ways to do a validation? Again, it is best to mimic split made by organizers and they split this by date. All before January, 1st, 2017 went to train, all after that date (2 months) - to test. The correct way to perform a split is with **sliding window**(credit for picture to Uber blogpost):
 

<img src="http://eng.uber.com/wp-content/uploads/2018/01/image3-4.png" width="500">




In [ ]:
from dateutil.relativedelta import relativedelta
import math

In [ ]:
sunspots = pd.read_csv("/content/drive/MyDrive/projector_course_data/sunspots_2014-2016.csv", parse_dates=['date'])
sunspots.head(3)

In [ ]:
sunspots.tail(3)

In [ ]:
sunspots['Month'] = sunspots["date"].dt.month
sunspots['Day'] = sunspots["date"].dt.day
sunspots['DayOfWeek'] = sunspots["date"].dt.dayofweek

In [ ]:
sunspots_train, sunspots_test = sunspots[sunspots["date"] < "2016-01-01"], sunspots[sunspots["date"]>="2016-01-01"]

In [ ]:
def create_validation(df, start_date):
    return df.loc[(df["date"] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df["date"] <  pd.to_datetime(start_date) + relativedelta(months=6))].index, \
           df.loc[(df["date"] >= pd.to_datetime(start_date) + relativedelta(months=6)) & \
                  (df["date"] <  pd.to_datetime(start_date) + relativedelta(months=12))].index

In [ ]:
train_dates = ["2014-01-01", "2014-07-01", "2015-01-01"]

In [ ]:
custom_cv = []
for train_date in train_dates:
    train_indicies, val_indicies = create_validation(sunspots_train, train_date)
    custom_cv.append((train_indicies, val_indicies))

In [ ]:
for train_indicies, val_indicies in custom_cv:
    print(min(train_indicies), min(val_indicies))

In [ ]:
sunspots_features_train = sunspots_train.drop(["value", "date"], axis = 1)
sunspots_label_train = sunspots_train["value"]

sunspots_features_test = sunspots_test.drop(["value", "date"], axis = 1)
sunspots_label_test = sunspots_test["value"]

In [ ]:
from sklearn.svm import SVR
regressor = SVR()

pipeline_r = Pipeline([("regressor", regressor)])
param_grid = [
  {"regressor__C": [0.01, 0.1, 1, 10, 100, 1000], "regressor__kernel": ["linear"]},
  {"regressor__C": [0.01, 0.1, 1, 10, 100, 1000], "regressor__gamma": [0.001, 0.0001], "regressor__kernel": ["rbf"]},
 ]
grid_search = GridSearchCV(pipeline_r, param_grid=param_grid, cv=custom_cv, scoring=metrics.make_scorer(metrics.mean_absolute_error))

%time grid_search.fit(sunspots_features_train, sunspots_label_train)

In [ ]:
grid_search.best_params_

In [ ]:
predictions = grid_search.predict(sunspots_features_test)

In [ ]:
grid_search.score(sunspots_features_test, sunspots_label_test)

### Group Validation

Group can refer to user id, store, city or any other entity. Another type of validation construction - is by group. Suppose you have a task to build a model to predict a weather in cities based on previous dates. Then if you know that in test set there are only new unseen cities, you should split yor dataset on train and validation such as there is no records for any city present in both train and validation.

In [ ]:
!pip uninstall pandas_profiling
!pip install pandas_profiling

In [ ]:
import pandas_profiling as pp
import pandas as pd
train = pd.read_csv("/content/drive/MyDrive/projector_course_data/train_titanic.csv")
test = pd.read_csv("/content/drive/MyDrive/projector_course_data/test_titanic.csv")
pp.ProfileReport(train)


In [ ]:
pp.ProfileReport(test)

## Metrics overview

Every competition can rely on different metrics that usually is dictated from business needs. It is important to understand the competition metric and optimize only this metric and not any other.

### <a name="3"></a>3. Classification metrics overview
Classification problems are probably the most common type of ML problem and as such there are many metrics that can be used to evaluate predictions for these problems. We will review some of them.

First note, that many of classifiers return soft labels, or scores for each class, such as probability, while others - hard labels i.e class where target belongs. Soft label can transformed to hard labels for example using threshold for binary classification

### <a name="3.1"></a>LogLoss

For binary classification, works with soft labels

$$ LogLoss = {-\frac{1}{N} \sum_{i=1}^{N}(y_{i}log(\hat{y_{i}})+(1-y_i)log(1-\hat{y_{i}}))}$$

### <a name="3.1"></a>Accuracy
Accuracy simply measures *what percent of your predictions were correct*. It's the ratio between the number of correct predictions and the total number of predictions. The downside is that it is hard to optimize and it cares about hard labels

$$accuracy = {\frac{\#\ correct}{\#\ predictions}}$$

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

X, y = train.drop('Survived', axis = 1), train.Survived
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42, stratify = y)

classifier = LogisticRegression()

pipeline = Pipeline([('classifier', classifier)])
model = pipeline.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [ ]:
# calculate accuracy
from sklearn import metrics
print(metrics.accuracy_score(y_test, y_pred))

Accuracy is also the most misused metric. It is really **only suitable** when there are an *equal number of observations in each class* (which is rarely the case) and that all *predictions and prediction errors are equally important*, which is often not the case.

### <a name="3.2"></a>Confusion Matrix
The confusion matrix is a handy presentation of the accuracy of a model with 2 or more classes. The table **presents predictions** on the x-axis and **accuracy outcomes** on the y-axis. The cells of the table are the number of predictions made by a machine learning algorithm.

In [ ]:
# first argument is true values, second argument is predicted values
# this produces a 2x2 numpy array (matrix)
conf = metrics.confusion_matrix(y_test, y_pred)
print(conf)

|                | Predicted Positive | Predicted  Negative|
|:--------------:|--------------------|--------------------|
| **Positive Cases** |      TP: 95      |      FN: 15      |
| **Negative Cases** |      FP: 20      |      TN: 49     |

- **True Positives (TP)**:
We correctly predicted that a person will survive: **95**
- **True Negatives (TN)**:
We correctly predicted that a person will not survive: **49**
- **False Positives (FP)**:
We incorrectly predicted that a person will survive: **20**
- **False Negatives (FN)**:
We incorrectly predicted that a person will not survive: **15**



Confusion matrix allows you to compute various classification metrics, and these metrics can guide your model selection. 

In [ ]:
# slice confusion matrix into four pieces for future use
TP = conf[1, 1]
TN = conf[0, 0]
FP = conf[0, 1]
FN = conf[1, 0]

You can learn more about the [Confusion Matrix on the Wikipedia article](https://en.wikipedia.org/wiki/Confusion_matrix).


### <a name="3.3"></a>Precision & Recall
Precision and recall are actually two metrics. But they are often used together.

**Precision** answers the question: *What percent of positive predictions were correct?*

$$precision = {\frac{\#\ true\ positive}{\#\ true\ positive + \#\ false\ positive}}$$

**Recall** answers the question: *What percent of the positive cases did you catch?*


$$recall = {\frac{\#\ true\ positive}{\#\ true\ positive + \#\ false\ negative}}$$

![](http://www.kdnuggets.com/images/precision-recall-relevant-selected.jpg)

See also a very good explanation of [Precision and recall](https://en.wikipedia.org/wiki/Precision_and_recall) in Wikipedia.

[To the table of contents](#0)

In [ ]:
# calculate precision
precision = TP / float(TP + FP)

assert precision == metrics.precision_score(y_test, y_pred)

In [ ]:
# calculate recall
recall = TP / float(TP + FN)

assert recall == metrics.recall_score(y_test, y_pred)

### <a name="3.4"></a>F1-score
The F1-score (sometimes known as the balanced F-beta score) is a single metric that combines both precision and recall via their harmonic mean:

$$F_1 = 2 {\frac{precision * recall}{precision + recall}}$$

Unlike the arithmetic mean, the harmonic mean tends toward the smaller of the two elements. Hence the F1 score will be small if either precision or recall is small.


In [ ]:
# calculate f1-score
f1 = 2 * precision * recall / (precision + recall)

assert f1 == metrics.f1_score(y_test, y_pred)

### <a name="3.4"></a>ROC AUC
The ROC curve is created by plotting the true positive rate (TPR) against the false positive rate (FPR) at various threshold settings.

More details here:
http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

![](https://i.stack.imgur.com/5x3Xj.png)



### <a name="3.5"></a>Classification Report
Scikit-learn does provide a convenience report when working on classification problems to give you a quick idea of the accuracy of a model using a number of measures.

The **classification_report()** function displays the precision, recall, f1-score and support for each class. (*support* is the number of occurrences of each class in *y_true*)

In [ ]:
# print a report on the binary classification problem
print(metrics.classification_report(y_test, y_pred))

### <a name="4"></a>4. Regression metrics overview


### <a name="3.1"></a>MSE (L2 Loss) and RMSE
MSE measures your mean square error from target:


$$ MSE = {\frac{1}{N} \sum_{i=1}^{N}(y_{i}-\hat{y_{i}})^2}$$

$$ RMSE = {\sqrt{MSE}}$$

RMSE and MSE is similiar in terms of minimizers - value minimizes RMSE **if and only if** it minimizes MSE. This means that in terms of competitions we can optimize MSE instead of RMSE. In fact it is easier to work with MSE. But there is a little bit of difference between the two for gradient-based models. The gradient of RMSE with respect to i-th prediction is basically equal to gradient of MSE multiplied by some value. The value doesn't depend on the index I. It means that travelling along MSE gradient is equivalent to traveling along RMSE gradient but with a different flowing rate and the flowing rate depends on MSE score itself. So, it is kind of dynamic.So even though RMSE and MSE are really similar in terms of models scoring, they can be not immediately interchangeable for gradient based methods. We will probably need to adjust some parameters like the learning rate.

To see model performance in terms of baseline mean usually R-squared is used. Or Adjusted R-squared to penalize for model parameters/features

$$ R^2 = {\frac{MSE}{\frac{1}{N} \sum_{i=1}^{N}(y_{i}-\bar{y_{i}})^2}}$$

R_squared is between 0 and 1.

In finance usually is used MAE metric

$$ MAE = {\frac{1}{N} \sum_{i=1}^{N}|y_{i}-\hat{y_{i}}|} $$

It is not differentiable in 0, but one can simply overcome that by coding simple *if else* condition. LightGBM can use MAE while xgboost **cannot**

If you care more about **relative** error  - **MSPE** or **MAPE** can be used. They are quite similar to MSE and MAE but incorporate error to relative values rather than absolute

If you care more about error for different values, you can apply a function to prediction and target before going into MSE. For example taking a **log(y+1)** will introduce (R)MSLE metric that penalizes more for mistakes for smaller number and less for larger

### <a name="4"></a>5. What to do with all these metrics?

#### OPTIMIZE

In fact there is often the case that model optimizes different metric from what you want it to optimize. Your possible actions are:
 + Find the model that optimizes your metric. LogLoss, MSE are present in most libraries
 + Create your own loss function and pass it to the model such as xgboost. You need to write your own derivatives
 + Preprocess your original target, for example use log(y+1) and RMSE instead of RMSLE
 + Postprocess your output predictions if you need accuracy
 + Run desired model with early stopping. This means optimize default loss, monitor your metric and stop training if you see your metric is not improving

## Final thoughts

* Try to reproduce the train/validation split (distribution) as you have in a test or on real-life data
* If there is a huge gap between test and real-life results, while train and val scores are similar - you have a leak in data. Carefully review your EDA. Try to remove most predictive features and compare the results
* Always try to look not only at F1 score but also at precision and recall (FP, FN) to find out when your model is wrong
* Holdout validation works very well when you have a lot of data points in Neural Networks for example
* Don't waste your time on building complex models to see if your validation is working. You should see this even submitting a constant value. Concentrate on very simple models such as linear/logistic regression for this
* If you have time and ran out of ideas, you can use the next trick - concatenate train and test sets. Create a variable that will have a value 'train' for examples in the train set or 'test' otherwise. Build a classifier that will try to predict whether an example belongs to either of two groups. * After that select the top examples with the highest probability to be included in the test and make them your validation.
* There is a possibility to GridSearch and compare algorithms using the statistical significance of Student criteria. The link to review this idea https://youtu.be/HT3QpRp2ewA?t=1071 

### Homework:
– Describe validation approaches and the best metrics for different datasets https://forms.gle/bQWy1Kfwf6UoJRMF8